In [ ]:
# initiate all functions

import requests
import json
import pandas as pd
import numpy as np


def get_response(username, type="chirp", limit=100):
    payload = json.dumps({"username": username, "limit": limit})
    headers = {
        "Content-Type": "application/json",
        "Cookie": "connect.sid=s%3AMykdjjS1VjZZOaQyIT7RhYG214TTMr8N.jIXuF6J9VFJrsGtqDp2Mfw41mBV14uGIyKbqXruSo68",
    }
    url = f"https://api.chirper.ai/v1/{type}"
    response = requests.request("GET", url, headers=headers, data=payload)
    if response.status_code == 200:
        # turn response text into json
        return json.loads(response.text)
    else:
        return {}


def get_chirps(username, limit=100):
    json_response = get_response(username, type="chirp", limit=limit)
    if len(json_response) == 0:
        return []
    else:
        return json_response["result"]["chirps"]

# Get language and sample chirps

In [ ]:
# get the unique chirpers
chirper_following_df = pd.read_csv("data/networks/full_following_network.csv")
all_unique_chirpers = list(
    set(chirper_following_df[["source", "target"]].values.flatten())
)
len(all_unique_chirpers)

In [ ]:
# create a chirper dataframe
chirper_df = pd.DataFrame(index=all_unique_chirpers, columns=["lang", "sample_chirps"])
searched_chirper_df = pd.read_csv("data/chirper_data.csv", index_col=0)
chirper_df.update(searched_chirper_df)

In [ ]:
# sample 10 chirps of each chirper, if it hasnt been done already
from random import sample

for idx, this_user in enumerate(all_unique_chirpers):
    if chirper_df.loc[this_user, "lang"] is not np.nan:
        if idx % 100 == 0:
            print(idx, end=" ")
        continue
    else:
        their_chirps = get_chirps(this_user, 100)

        if len(their_chirps) == 0:
            chirper_df.loc[this_user, "lang"] = "NO RESPONSE"
            chirper_df.loc[this_user, "sample_chirps"] = "NO RESPONSE"
        else:
            their_lang = (
                pd.Series([str(chirp["lang"]) for chirp in their_chirps])
                .value_counts()
                .keys()[0]
            )
            if their_lang in ["None", "nan", "", "NA", "NaN"]:
                their_lang = "NO RESPONSE"

            if len(their_chirps) > 10:
                their_chirps = sample(their_chirps, 10)
            their_chirps = [chirp["content"] for chirp in their_chirps]

            chirper_df.loc[this_user, "lang"] = their_lang
            chirper_df.loc[this_user, "sample_chirps"] = " ".join(their_chirps)
        if idx % 100 == 0:
            print(idx, end=" ")

In [ ]:
# save to csv
chirper_df = chirper_df.reset_index().rename(columns={"index": "chirper"})
chirper_df.to_csv('data/chirper_data.csv', index=False)
chirper_df

In [ ]:
chirper_df.lang.value_counts()